<a href="https://colab.research.google.com/github/Rogerio-mack/IA_2024S1/blob/main/AI_N1A_Lab_2024S1_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

<br>
<br>
<br>
<br>
<br>

# N1A Lab Caso: **Controle de Qualidade de Peças de uma Indústria**
---

**pieces** é um data set para o controle de qualidade de peças produzidas por uma indústria. São empregadas quatro medidas (A, B, C e D) para o controle da qualidade das peças. As peças são então classificadas como *Accept, Refurbish* ou *Reject* segundo o controle de qualidade (atributo `Quality`). A indústria ainda conta com 3 unidades para a produção dessas peças (atributo `Unit`).

Seu trabalho é criar modelos supervisionados de classificação para predição da qualidade das peças e em seguida você empregar um modelo de regressão linear para predição de um das medidas.


# Q0. Aquisição dos Dados

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces3.csv')
df = df.dropna() # elimina dados ausentes
display(df.head())

X_new = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces3_new.csv')
display(X_new.head())

,id,A,B,C,D,Quality,Unit
0,168,5.0,3.5,1.3,0.3,Reject,BH
1,527,6.9,3.1,5.4,2.1,Refurbish,BH
2,493,6.8,3.0,5.5,2.1,Refurbish,SP
3,584,5.9,3.0,5.1,1.8,Refurbish,SP
5,299,5.5,2.3,4.0,1.3,Accept,RJ


,A,B,C,D,Unit
0,6.1,2.8,4.7,1.2,BH
1,5.5,2.6,4.4,1.2,SP
2,7.7,3.0,6.1,2.3,BH
3,7.2,3.2,6.0,1.8,BH
4,4.6,3.4,1.4,0.3,RJ


# Q1.

**Qual percentual de qualidade de peças apresenta o menor valor e qual é esse valor?**



In [ ]:
print('% by Quality:\n', df.Quality.value_counts() / len(df))

% by Quality:
 Quality
Reject       0.355872
Refurbish    0.343416
Accept       0.300712
Name: count, dtype: float64


# Q2.

Crie um modelo de **classificação logística** da qualidade das peças **a partir somente dos atributos numéricos que fazem sentido para o modelo**.

1. Separe os dados de treinamento e teste estratificados com 30% dos dados para teste e `random_state=1`.
2. Empregue o máximo de 1000 iterações para o ajuste do modelo.
2. Não empregue outros parâmetros não solicitados.

**Qual a acuracidade obtida pelo modelo de Regressão Logística?**



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# 1. Variáveis preditoras e objetivo
X = df[['A', 'B', 'C', 'D']]
y = df['Quality']

# 2. Separação Treinamento e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

# 3. Definição do Modelo
clf = LogisticRegression(max_iter=1000)

# 4. Treinamento
clf.fit(X_train, y_train)

# 5. Predição X_test e métricas
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(clf, '\n', accuracy)

print('\n', classification_report(y_test, y_pred))



LogisticRegression(max_iter=1000) 
 0.9704142011834319

               precision    recall  f1-score   support

      Accept       0.96      0.94      0.95        51
   Refurbish       0.95      0.97      0.96        58
      Reject       1.00      1.00      1.00        60

    accuracy                           0.97       169
   macro avg       0.97      0.97      0.97       169
weighted avg       0.97      0.97      0.97       169



# Q3.

**Qual classe apresentou maior número absoluto de falsos positivos?**

A classe refurbish, com uma precisão de 0.95 (3 falso positivos).

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('\n', cm)


 [[48  3  0]
 [ 2 56  0]
 [ 0  0 60]]


# Q4.

**Qual a predição do modelo para os novos casos X_new?**

In [ ]:
print( clf.predict(X_new[['A','B','C','D']]) )


['Accept' 'Accept' 'Refurbish' 'Refurbish' 'Reject' 'Reject']


# Q5.

Crie um modelo de Regressão Linear para o Cálculo da medida `C` das peças com base nos atributos `A`, `B`, `D` e `Unit`.

**Qual o coeficiente de determinação obtido pelo modelo e qual atributo se mostra não significativo?**

In [ ]:
import statsmodels.formula.api as sm

model = sm.ols(formula='C ~ A + B + D + Unit', data=df.drop(columns='id'))
result = model.fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      C   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                     3707.
Date:                Sun, 07 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:47:49   Log-Likelihood:                -139.36
No. Observations:                 562   AIC:                             290.7
Df Residuals:                     556   BIC:                             316.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7594      0.152     -4.988      0.0

# Q6.

Aplique o modelo de Regressão obtido para os dados `X_new`.

**Qual o erro absoluto médio (MAE) obtido das previsões de X_new?**

$$MAE= \frac{1}{n} \sum_{i=1}^n | (y_i - y_{pred_i}) | $$

In [ ]:
y_pred = result.predict(X_new)

print(np.abs(y_pred - X_new.C).mean())

0.447213911506807
